# Introduction
General housekeeping for running the notebooks and analysis pipeline.

In [1]:
import pickle as pkl
import scanpy as sc
import matplotlib as mpl

In [2]:
mountpoint = '/data/clue/'

Path to the most recent concat object:

In [3]:
path = mountpoint + 'prod/comb/h5ads/concat_5.h5ad'

# Create Metadata Dictionary

I use a `pkl` called `meta.pkl` in various notebooks throughout the processing. Here is where I create it.

In [21]:
meta = dict()

## `cts`

In [22]:
concat = sc.read_h5ad(path)

In [23]:
ct_types = [i for i in concat.obs.columns if i.startswith('ct')]

In [24]:
meta['cts'] = dict()
for ct_type in ct_types:
    meta['cts'][ct_type] = concat.obs[ct_type].cat.categories.tolist()

## `ct` colors

In [25]:
ct_colors = dict()
for ct_type in ct_types:
    ct_colors[ct_type] = dict()
    ct_colors[ct_type]['hex'] = dict(zip(concat.obs[ct_type].cat.categories, concat.uns[ct_type + '_colors']))
    ct_colors[ct_type]['rgb'] = dict(zip(concat.obs[ct_type].cat.categories, list(map(mpl.colors.hex2color, concat.uns[ct_type + '_colors']))))

In [26]:
meta['ct_colors'] = ct_colors

## `conds`

In [27]:
meta['conds'] = dict()
meta['conds']['conds_all'] = ['0', 'A', 'B', 'C', 'G', 'P' ,'R']
meta['conds']['conds_filt'] = ['A', 'B', 'C', 'G', 'P' ,'R']
meta['conds']['conds_focus'] = ['A', 'B', 'C', 'G' ,'R']

## `cond_colors`

In [28]:
cond_colors = dict()
conds_all = meta['conds']['conds_all'] # 
cond_colors['hex'] = dict(zip(conds_all, sc.pl.palettes.default_20))
cond_colors['rgb'] = dict(zip(conds_all, list(map(mpl.colors.hex2color, sc.pl.palettes.default_20))))

In [29]:
meta['cond_colors'] = cond_colors

In [35]:
# with open(mountpoint + 'meta.pkl', 'wb') as file:
#     pkl.dump(meta, file)

In [36]:
with open(mountpoint + 'meta.pkl', 'rb') as file:
    meta = pkl.load(file)